# Run Topas from Jupyter
* Version 1.7
* Last update: 2017/08/17
* Created by Anders Bank Blichfeld and Susanne Linn Skjærvø. Modified by Ola Gjønnes Grendal (version 1.7) for doing 
* backwards sequential refinements.
--------------------------------------------------------------------------------------------------------------------------------
1. Save this notebook anywhere
2. Make a folder in this directory with the datafile extension as a name (i.e. xye, dat, raw etc.) and put all datafiles in it. 
3. Put the .inp file in the directory above
4. Replace the 'directory+filename' in the .inp file with "IN_FILE.xye". Add "DATASET" wherever you want only the filename to be rendered (i.e. same as for batch refinements with 'topas_batch_initial.bat').  
5. Make sure all directories and files are listed correctly below. 
6. Run the script.


- To be able to generate plots of the refined patterns, a macro called "macro Out_X_Yobs_Ycalc_Difference(file)" must be added to 'C:/Topas5/local.inc', and then added as one of the output parameters in the .inp file. Ask Susanne or Ola for details.

- The current script should work for all files with namestyle: as_long_as_you_want_12345.xye

- It should be noted that there is a difference between version 2.x and 3.x of Python in how the Topas output code is rendered. While version 2 will output ascii, version 3 will give you bytes. The current script should work for both, but if you are having problems write 'line' instead of 'line.decode('ascii')'.

- DIRECTORY MUST BE WITHOUT SPACES

In [1]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import time as t
import pylab as pl
from IPython import display
import os, subprocess, re, shutil
import numpy as np

In [33]:
presidents = ["Washington", "Adams", "Jefferson", "Madison", "Monroe", "Adams", "Jackson"]
for num, name in enumerate(presidents, start=1):
    print(presidents[num])

Adams
Jefferson
Madison
Monroe
Adams
Jackson


IndexError: list index out of range

## Directories and files

#### User defined

In [53]:
Inp_file='Test_comp_batch.inp'                                                  # name of input file
ext = 'xye'                                                                # datafile extension
directory1='C:/Documents/PhD/Resultater/In-situ/5_SNBL_feb_2018/Test_backwards_refinement/'                                                              # working directory
#directory1=os.getcwd()+''                          # if you want the directory of the notebook to define working space (does not work for UNC paths)
directory3=directory1+'INP_OUT/'                                            # folder for output files
out_file='Backward_refinement.txt'                #Set name of output file with all refined values

In [54]:
directory2=directory1+'%s/'%(ext)                                          # folder of datafiles

if not os.path.exists(os.path.dirname(directory3)):
    os.makedirs(os.path.dirname(directory3))                               # make directory for output

## Process parameters

#### User defined

In [57]:
plotfig = 0                       # 1: graphical output (makes process slower), 0: text output
printall = 0                      # 1: print full refinement output, 0: print only most essential
pause = 0                         # 1: if you want extra time to look at output of each refinement (problems with stopping loop: click 'restart & clear output' in kernel dropdown menu)

start = 0                        # Define the filenumbers you want to refine here
end = 913

keywords=['DATASET', 'IN_FILE']   # keywords you want to replace in your .inp file, make sure pattern_str and replace_str has equally many arguments

In [58]:
all_files = directory2
all_data=[]
for filename in os.listdir(all_files):
    if filename.endswith(".%s"%(ext)): 
        all_data+=[all_files+filename]
        continue
    else:
        continue
all_data.sort(reverse=True)                  #Set False for starting with first file, True for starting with last file
print("Number of datafiles files found: %s"%len(all_data))
all_data=all_data[start:end]
print("Refining this many files: %s"%len(all_data))

Number of datafiles files found: 913
Refining this many files: 913


## Run Topas

In [59]:
counter=0

for count, files in enumerate(all_data):
    if counter != 0:
        Base1=os.path.splitext(os.path.basename(files))[0]                 # filenames of current the datafiles without the extension
        Base2=os.path.splitext(os.path.basename(all_data[count-1]))[0]    #filenames of previous 
        temp_inp=directory3+Base1+os.path.splitext(directory1+Inp_file)[1]    # temporary variable making individual inputfiles for every datafile
        shutil.copy(directory3+Base2+'.out',temp_inp)                         # copies the .inpfile in directory1 to directory3 and gives it the same name as the datafile 
        print('   '+Base1+'.inp')
        pattern_str=[Base2]                                   
        replace_str=[Base1]                                  # this is what you want to replace the keywords with (make sure patter_str and replace_str has the same amounts of arguments)
        for i,word in enumerate(pattern_str):                                # this loop replaces all the keywords you have set
            with open(temp_inp) as f:
                s = f.read()
            
            with open(temp_inp,'w') as f:
                s = s.replace(pattern_str[i],replace_str[i])
                f.write(s)
                
        cmd = 'C:/TOPAS5/tc %s' %(temp_inp)                                  # setting inputparameter for command prompt
        
    
        process = subprocess.Popen(cmd,stdout=subprocess.PIPE)               # launching Topas from the command prompt
        line_save=[]
        for line in process.stdout:                                          # retrieves certain parts of the refinement output                        
            if re.search(b'seconds',line):
                if plotfig != 1 and printall !=1:
                    print('   '+' '+line_save.decode('ascii'))
                else:
                    no.append(int(os.path.splitext(os.path.basename(files))[0][35:])) #First bracket: 0=filename, 1=fileextension. Second bracket: first number removes start of filename, second number removes end of filename
                    Rwp.append(float(line_save.decode('ascii')[24:33]))
                    time.append(float(line_save.decode('ascii')[12:17]))
                    
            line_save=line
    if counter == 0:
        Base=os.path.splitext(os.path.basename(files))[0]                 # filenames of the datafiles without the extension
        temp_inp=directory3+Base+os.path.splitext(directory1+Inp_file)[1]    # temporary variable making individual inputfiles for every datafile
        shutil.copy(directory1+Inp_file,temp_inp)                            # copies the .inpfile in directory1 to directory2 and gives it the same name as the datafile 
        print('   '+Base+'.inp')
        pattern_str=keywords                                   
        replace_str=[Base, directory2+Base]                                  # this is what you want to replace the keywords with (make sure patter_str and replace_str has the same amounts of arguments)
        for i,word in enumerate(pattern_str):                                # this loop replaces all the keywords you have set
            with open(temp_inp) as f:
                s = f.read()
            
            with open(temp_inp,'w') as f:
                s = s.replace(pattern_str[i],replace_str[i])
                f.write(s)
                
        cmd = 'C:/TOPAS5/tc %s' %(temp_inp)                                  # setting inputparameter for command prompt

    
        process = subprocess.Popen(cmd,stdout=subprocess.PIPE)               # launching Topas from the command prompt
        line_save=[]
        for line in process.stdout:                                          # retrieves certain parts of the refinement output                        
            if re.search(b'seconds',line):
                if plotfig != 1 and printall !=1:
                    print('   '+' '+line_save.decode('ascii'))
                else:
                    no.append(int(os.path.splitext(os.path.basename(files))[0][35:])) #First bracket: 0=filename, 1=fileextension. Second bracket: first number removes start of filename, second number removes end of filename
                    Rwp.append(float(line_save.decode('ascii')[24:33]))
                    time.append(float(line_save.decode('ascii')[12:17]))
                    
            line_save=line
        counter=counter+1
        

from IPython.display import Audio, display

def allDone():
  display(Audio('C:/Documents/PhD/Diverse/Alesis-Sanctuary-QCard-Loose-Bell-C5.wav', autoplay=True))
allDone()
print('FINISHED!')

   SBN_40_T225_P200_Nbaged_heat_0001p_00912.inp
      4  Time   0.08  Rwp    2.160   -0.000 MC   0.45 1

   SBN_40_T225_P200_Nbaged_heat_0001p_00911.inp
      2  Time   0.07  Rwp    2.163    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00910.inp
      2  Time   0.07  Rwp    2.162    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00909.inp
      2  Time   0.09  Rwp    2.164    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00908.inp
      3  Time   0.09  Rwp    2.165    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00907.inp
      2  Time   0.09  Rwp    2.163    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00906.inp
      3  Time   0.10  Rwp    2.161    0.000 MC  100.00 4

   SBN_40_T225_P200_Nbaged_heat_0001p_00905.inp
      3  Time   0.09  Rwp    2.158   -0.000 MC   0.00 1

   SBN_40_T225_P200_Nbaged_heat_0001p_00904.inp
      3  Time   0.07  Rwp    2.160   -0.000 MC   0.00 1

   SBN_40_T225_P200_Nbaged_heat_0001p_00903.inp
 

FINNISHED!
